<a href="https://colab.research.google.com/github/babsubra1980/Term-Project-ParallelAlgorithm/blob/main/bellmanFord.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [2]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-s3k72u9r
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-s3k72u9r
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4294 sha256=14958e28b153e56ae7b154b64391ab3e3c5145c3889ee46f8b30717cac9afd5f
  Stored in directory: /tmp/pip-ephem-wheel-cache-racdbj7j/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [3]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [8]:

%%cu
#include <cuda_runtime.h>
#include <device_launch_parameters.h>
#include <iostream>

#define MAX_INT 1000000

__global__ void bellmanFord(int *dev_edges, int *dev_weights, int *dev_distances, int num_vertices, int num_edges) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;

    if (tid < num_vertices) {
        for (int i = 0; i < num_edges; ++i) {
            int source = dev_edges[i * 3];
            int destination = dev_edges[i * 3 + 1];
            int weight = dev_weights[i];

            if (dev_distances[source] != MAX_INT && dev_distances[source] + weight < dev_distances[destination]) {
                atomicMin(&dev_distances[destination], dev_distances[source] + weight);
            }
        }
    }
}

int main() {
    const int num_vertices = 5;
    const int num_edges = 8;

    int edges[num_edges][3] = {
        {0, 1, 5},
        {0, 2, -2},
        {1, 2, 4},
        {1, 3, 3},
        {2, 1, 6},
        {2, 3, 7},
        {2, 4, 2},
        {3, 4, 1}
    };

    int weights[num_edges] = {5, -2, 4, 3, 6, 7, 2, 1};

    int distances[num_vertices] = {0, MAX_INT, MAX_INT, MAX_INT, MAX_INT};

    int *dev_edges, *dev_weights, *dev_distances;

    cudaMalloc((void**)&dev_edges, num_edges * 3 * sizeof(int));
    cudaMalloc((void**)&dev_weights, num_edges * sizeof(int));
    cudaMalloc((void**)&dev_distances, num_vertices * sizeof(int));

    cudaMemcpy(dev_edges, edges, num_edges * 3 * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(dev_weights, weights, num_edges * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(dev_distances, distances, num_vertices * sizeof(int), cudaMemcpyHostToDevice);

    const int num_blocks = 1;
    const int num_threads_per_block = num_vertices;

    bellmanFord<<<num_blocks, num_threads_per_block>>>(dev_edges, dev_weights, dev_distances, num_vertices, num_edges);

    cudaMemcpy(distances, dev_distances, num_vertices * sizeof(int), cudaMemcpyDeviceToHost);

    // Print the distances
    for (int i = 0; i < num_vertices; ++i) {
        std::cout << "Distance to vertex " << i << " is: " << distances[i] << std::endl;
    }

    cudaFree(dev_edges);
    cudaFree(dev_weights);
    cudaFree(dev_distances);

    return 0;
}


Distance to vertex 0 is: 0
Distance to vertex 1 is: 4
Distance to vertex 2 is: -2
Distance to vertex 3 is: 5
Distance to vertex 4 is: 0

